In [2]:
# Read in the bikeshare data to a pandas dataframe just as you did for the previous assignment.
# Split the dataframe horizontally into two separate dataframes each having N/2 rows. 
# Call the dataframe with first N/2 rows bikeshare_train, and the latter half bikeshare_test
# Using the bikeshare_train dataframe run a regression with cnt as our target 
# variable, and temp, atemp, hum, workingday, hour, weathersit as the feature variables. 
# Specify the R^2 value. We'll call this our training model 
# The resulting model should have one more coefficient than the number of feature variables.
# Describe the error (in terms of R^2) between the model you just calculated and the bikeshare_test. 
# Is this higher or lower than the R^2 of the previous step? What's the reason for this difference?
# Create a binary variable for rush hour defined by 6-9am & 4-6pm.
# Run the regression again. Using a similar process used in steps 2 and 3, 
# assert whether this improves the results or not. Why? (You should be able to explain this quantitatively)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


%matplotlib inline

url = '../../../DAT_20_NYC/Data/bikeshare.csv'
# Read in the bikeshare data to a pandas dataframe just as you did for the previous assignment.
bikeshare_csv = pd.read_csv(url)

In [15]:
bikeshare_csv.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1


In [16]:
features = ['temp','atemp','hum','workingday','hr','weathersit']
# Split the dataframe horizontally into two separate dataframes each having N/2 rows. 
# Call the dataframe with first N/2 rows bikeshare_train, and the latter half bikeshare_test
bikeshare_train = bikeshare_csv[:(len(bikeshare_csv) / 2)]
bikeshare_test = bikeshare_csv[(len(bikeshare_csv) / 2):]

print "Original DF: %s" % len(bikeshare_csv) + " Rows"
print "Train: %s" % len(bikeshare_train) + " Rows"
print "Test: %s" % len(bikeshare_test) + " Rows"

Original DF: 17379 Rows
Train: 8689 Rows
Test: 8690 Rows


In [20]:
# Using the bikeshare_train dataframe run a regression with cnt as our target 
# variable, and temp, atemp, hum, workingday, hour, weathersit as the feature variables. 

import statsmodels.formula.api as smf

# create a fitted model in one line
model = smf.ols(formula='cnt ~ temp + atemp + hum + workingday + hr + weathersit', data=bikeshare_train).fit()

# Specify the R^2 value. We'll call this our training model 
print "R^2 is: %s" % model.rsquared

R^2 is: 0.36711033766


In [6]:
# Using the bikeshare_train dataframe run a regression with cnt as our target 
# variable, and temp, atemp, hum, workingday, hour, weathersit as the feature variables. 

# X_train = bikeshare_train[features]
# X_test = bikeshare_test[features]
# y_train = bikeshare_train['cnt']
# y_test = bikeshare_test['cnt']

# model = LinearRegression()

# model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [18]:
# Specify the R^2 value. We'll call this our training model 
# The resulting model should have one more coefficient than the number of feature variables.
# but it doesn't :( 
# print "R^2: %s" % model.score(X_train, y_train)
# print "Intercept: %s" % model.intercept_
# #print coefficients
# zip(X_train.columns, model.coef_)

In [8]:
# Describe the error (in terms of R^2) between the model you just calculated and the bikeshare_test. 
# Is this higher or lower than the R^2 of the previous step? What's the reason for this difference?
print "R^2: %s" % model.score(X_test, y_test)

# must be overfitting the training dataset

R^2: 0.139406068786


In [9]:
# Create a binary variable for rush hour defined by 6-9am & 4-6pm.
# Run the regression again. Using a similar process used in steps 2 and 3, 
# assert whether this improves the results or not. Why? (You should be able to explain this quantitatively
from math import ceil
rush_hour = [int((x > 5 and x < 10) or (x > 15 and x < 19)) for x in bikeshare_csv['hr']]

X1_train = X_train
X1_train['rush_hour'] = rush_hour[ : 8689]
X1_test = bikeshare_test[features]
X1_test['rush_hour'] = rush_hour[8689 : ]
y1_train = bikeshare_train['cnt']
y1_test = bikeshare_test['cnt']

model = LinearRegression()

model.fit(X1_train,y1_train)


/Users/corey/.virtualenvs/GA/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/corey/.virtualenvs/GA/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [12]:
print "R^2: %s" % model.score(X1_train, y1_train)
print "Intercept: %s" % model.intercept_
# print coefficients
zip(X1_train.columns, model.coef_)

R^2: 0.505642825624
Intercept: -10.0675882001


[('temp', 124.27058163381531),
 ('atemp', 169.23306331263109),
 ('hum', -120.99052840738021),
 ('workingday', -1.0337982964049584),
 ('hr', 6.0665799133984413),
 ('weathersit', -7.0620522184271799),
 ('rush_hour', 109.1759659178276)]

In [ ]:
print "R^2: %s" % model.score(X1_test, y1_test)

In [ ]:
# the model has improved, because the R^2 value difference on cross validation improved by ~.14